## 1. Setup

In [ ]:
from gqlalchemy import Memgraph
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np

# Connect to MemGraph
memgraph = Memgraph(host='127.0.0.1', port=7687)
memgraph.drop_database()

print("[OK] Connected to MemGraph")

## 2. Create a Social Network

Let's create a more complex social network for algorithm demonstrations.

In [ ]:
# Create users
users = [
    {"id": 1, "name": "Alice", "city": "New York"},
    {"id": 2, "name": "Bob", "city": "San Francisco"},
    {"id": 3, "name": "Charlie", "city": "Seattle"},
    {"id": 4, "name": "Diana", "city": "Boston"},
    {"id": 5, "name": "Eve", "city": "Austin"},
    {"id": 6, "name": "Frank", "city": "Chicago"},
    {"id": 7, "name": "Grace", "city": "Miami"},
    {"id": 8, "name": "Henry", "city": "Denver"},
    {"id": 9, "name": "Iris", "city": "Portland"},
    {"id": 10, "name": "Jack", "city": "Phoenix"}
]

for user in users:
    query = f"""
    CREATE (u:User {{id: {user['id']}, name: '{user['name']}', city: '{user['city']}'}});
    """
    memgraph.execute(query)

print(f"[OK] Created {len(users)} users")

In [ ]:
# Create friendships (bidirectional)
friendships = [
    (1, 2), (1, 3), (1, 4),
    (2, 3), (2, 5),
    (3, 4), (3, 6),
    (4, 5), (4, 7),
    (5, 6), (5, 8),
    (6, 7), (6, 9),
    (7, 8), (7, 10),
    (8, 9),
    (9, 10)
]

for user1_id, user2_id in friendships:
    query = f"""
    MATCH (u1:User {{id: {user1_id}}}), (u2:User {{id: {user2_id}}})
    CREATE (u1)-[:FRIENDS_WITH]->(u2),
           (u2)-[:FRIENDS_WITH]->(u1);
    """
    memgraph.execute(query)

print(f"[OK] Created {len(friendships)} bidirectional friendships")

## 3. Degree Centrality

Find the most connected users in the network.

In [ ]:
# Calculate degree centrality (number of connections)
query = """
MATCH (u:User)-[r:FRIENDS_WITH]-()
RETURN u.name as user, u.city as city, count(r) as degree
ORDER BY degree DESC;
"""
results = memgraph.execute_and_fetch(query)

df = pd.DataFrame([dict(row) for row in results])
print("Degree Centrality (Most Connected Users):")
print(df)

# Visualize
plt.figure(figsize=(10, 6))
plt.bar(df['user'], df['degree'], color='skyblue')
plt.xlabel('User')
plt.ylabel('Number of Connections')
plt.title('Degree Centrality by User')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## 4. Shortest Path Analysis

Find the shortest paths between users.

In [ ]:
# Find shortest path between two specific users
query = """
MATCH path = shortestPath(
    (u1:User {name: 'Alice'})-[:FRIENDS_WITH*]-(u2:User {name: 'Jack'})
)
RETURN [node in nodes(path) | node.name] as path_names,
       length(path) as path_length;
"""
results = list(memgraph.execute_and_fetch(query))

if results:
    result = results[0]
    print(f"Shortest path from Alice to Jack:")
    print(f"Path: {' -> '.join(result['path_names'])}")
    print(f"Length: {result['path_length']} hops")

In [ ]:
# Calculate average shortest path length for the entire network
query = """
MATCH (u1:User), (u2:User)
WHERE id(u1) < id(u2)
MATCH path = shortestPath((u1)-[:FRIENDS_WITH*]-(u2))
RETURN avg(length(path)) as avg_path_length,
       min(length(path)) as min_path_length,
       max(length(path)) as max_path_length;
"""
results = list(memgraph.execute_and_fetch(query))

if results:
    result = results[0]
    print("Network Path Statistics:")
    print(f"Average path length: {result['avg_path_length']:.2f}")
    print(f"Minimum path length: {result['min_path_length']}")
    print(f"Maximum path length: {result['max_path_length']}")
    print(f"\nThis represents the 'small world' property of the network.")

## 5. Triangle Counting

Count triangles in the network (closed triplets of friends).

In [ ]:
# Count triangles per user
query = """
MATCH (u:User)-[:FRIENDS_WITH]-(friend1:User)-[:FRIENDS_WITH]-(friend2:User)-[:FRIENDS_WITH]-(u)
WHERE id(u) < id(friend1) AND id(friend1) < id(friend2)
RETURN u.name as user, count(*) as triangles
ORDER BY triangles DESC;
"""
results = memgraph.execute_and_fetch(query)

df = pd.DataFrame([dict(row) for row in results])
print("Triangle Count per User:")
print(df)
print(f"\nTriangles indicate strongly connected groups of friends.")

## 6. Clustering Coefficient

Measure how clustered the network is.

In [ ]:
# Calculate local clustering coefficient for each user
query = """
MATCH (u:User)
OPTIONAL MATCH (u)-[:FRIENDS_WITH]-(friend)
WITH u, collect(DISTINCT friend) as friends, count(DISTINCT friend) as degree
WHERE degree > 1
UNWIND friends as f1
UNWIND friends as f2
WITH u, friends, degree, f1, f2
WHERE id(f1) < id(f2)
OPTIONAL MATCH (f1)-[:FRIENDS_WITH]-(f2)
WITH u, degree, count(DISTINCT f2) as connected_pairs
WITH u, degree, connected_pairs, 
     (degree * (degree - 1)) / 2.0 as possible_pairs
RETURN u.name as user, 
       degree,
       connected_pairs,
       possible_pairs,
       CASE WHEN possible_pairs > 0 
            THEN toFloat(connected_pairs) / possible_pairs 
            ELSE 0.0 
       END as clustering_coefficient
ORDER BY clustering_coefficient DESC;
"""
results = memgraph.execute_and_fetch(query)

df = pd.DataFrame([dict(row) for row in results])
print("Clustering Coefficient (How tightly knit each user's friend group is):")
print(df)
print(f"\nHigher coefficient = friends are more connected to each other")

## 7. Community Detection (Manual)

Identify potential communities by analyzing connection patterns.

In [ ]:
# Find users with shared cities (simple community detection)
query = """
MATCH (u:User)
WITH u.city as city, collect(u.name) as users
WHERE size(users) > 1
RETURN city, users, size(users) as community_size
ORDER BY community_size DESC;
"""
results = memgraph.execute_and_fetch(query)

df = pd.DataFrame([dict(row) for row in results])
print("Geographic Communities:")
print(df)

## 8. Betweenness Centrality (Simplified)

Find users who act as bridges between different parts of the network.

In [ ]:
# Count how many shortest paths pass through each user
query = """
MATCH (source:User), (target:User)
WHERE id(source) < id(target)
MATCH path = shortestPath((source)-[:FRIENDS_WITH*]-(target))
WITH nodes(path) as path_nodes
UNWIND path_nodes as node
WITH node
WHERE 'User' IN labels(node)
RETURN node.name as user, count(*) as paths_through
ORDER BY paths_through DESC;
"""
results = memgraph.execute_and_fetch(query)

df = pd.DataFrame([dict(row) for row in results])
print("Betweenness Centrality (Bridge Users):")
print(df.head(10))
print(f"\nUsers with high scores act as bridges in the network")

## 9. Network Visualization with Metrics

In [ ]:
# Get network data
query = """
MATCH (u1:User)-[r:FRIENDS_WITH]->(u2:User)
WHERE id(u1) < id(u2)
RETURN u1.name as source, u2.name as target;
"""
results = memgraph.execute_and_fetch(query)

# Get degree data
degree_query = """
MATCH (u:User)-[r:FRIENDS_WITH]-()
RETURN u.name as user, count(r) as degree;
"""
degree_results = memgraph.execute_and_fetch(degree_query)
degrees = {row['user']: row['degree'] for row in degree_results}

# Create NetworkX graph
G = nx.Graph()
for row in results:
    data = dict(row)
    G.add_edge(data['source'], data['target'])

# Visualize
plt.figure(figsize=(14, 10))
pos = nx.spring_layout(G, k=2, iterations=50, seed=42)

# Node sizes based on degree
node_sizes = [degrees.get(node, 1) * 500 for node in G.nodes()]

# Draw network
nx.draw_networkx_nodes(G, pos, node_color='lightblue', 
                       node_size=node_sizes, alpha=0.9)
nx.draw_networkx_edges(G, pos, edge_color='gray', alpha=0.5)
nx.draw_networkx_labels(G, pos, font_size=10, font_weight='bold')

plt.title("Social Network (Node size = Degree Centrality)", fontsize=16)
plt.axis('off')
plt.tight_layout()
plt.show()

# Network statistics
print(f"\nNetwork Statistics:")
print(f"Nodes: {G.number_of_nodes()}")
print(f"Edges: {G.number_of_edges()}")
print(f"Average degree: {sum(dict(G.degree()).values()) / G.number_of_nodes():.2f}")
print(f"Density: {nx.density(G):.3f}")
print(f"Is connected: {nx.is_connected(G)}")

## 10. Friend Recommendations

Suggest potential friends based on mutual connections.

In [ ]:
# Find friend recommendations for a specific user
query = """
MATCH (u:User {name: 'Alice'})-[:FRIENDS_WITH]-(friend)-[:FRIENDS_WITH]-(recommended:User)
WHERE NOT (u)-[:FRIENDS_WITH]-(recommended) AND u <> recommended
WITH recommended, count(DISTINCT friend) as mutual_friends
RETURN recommended.name as recommendation, 
       recommended.city as city,
       mutual_friends
ORDER BY mutual_friends DESC;
"""
results = memgraph.execute_and_fetch(query)

df = pd.DataFrame([dict(row) for row in results])
print("Friend Recommendations for Alice:")
print(df)
print(f"\nRecommendations based on mutual friends")

## 11. Connected Components

Check if the network has isolated groups.

In [ ]:
# Using NetworkX to find connected components
components = list(nx.connected_components(G))

print(f"Number of connected components: {len(components)}")
print(f"\nComponent sizes:")
for i, component in enumerate(components, 1):
    print(f"Component {i}: {len(component)} nodes - {sorted(component)}")

if len(components) == 1:
    print(f"\n[OK] The entire network is connected!")
else:
    print(f"\n[WARNING] The network has {len(components)} isolated groups")

## Summary

In this notebook, you learned about:
1. Degree Centrality - Finding the most connected users
2. Shortest Path Analysis - Understanding network distances
3. Triangle Counting - Identifying tightly-knit groups
4. Clustering Coefficient - Measuring network cohesion
5. Community Detection - Finding groups within the network
6. Betweenness Centrality - Identifying bridge nodes
7. Friend Recommendations - Practical applications
8. Connected Components - Network connectivity analysis

These algorithms are fundamental for social network analysis, recommendation systems, and understanding network structure!